In [1]:
import requests
import json
import pandas as pd

from datetime import datetime, timedelta
import sqlalchemy

In [2]:
def get_weather(city):
    APIKey = '6b85c8e338c78a1d06d8155b22c223ef'
    
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={APIKey}&units=metric"
    response = requests.get(url)

    weather = response.json()['list']
    weather = pd.json_normalize(weather)

    weather = weather[['dt_txt', 'pop', 'main.temp', 'wind.speed']]
    weather['dt_txt'] = pd.to_datetime(weather['dt_txt'])

    weather.columns = ['time_weather', 'pop', 'temp', 'wind_s']
    
    weather['pop'] = pd.to_numeric(weather['pop'])
    weather['temp'] = pd.to_numeric(weather['temp'])
    weather['wind_s'] = pd.to_numeric(weather['wind_s'])

    tomorrow = datetime.now().date() + timedelta(days=1)
    thedayafter = datetime.now().date() + timedelta(days=2)

    weather = weather.loc[weather['time_weather'] > str(tomorrow)]
    weather = weather.loc[weather['time_weather'] < str(thedayafter)]
    
    weather = weather.max()

    weather = weather.to_frame().transpose()
    weather['pop'] = pd.to_numeric(weather['pop'])
    weather['temp'] = pd.to_numeric(weather['temp'])
    weather['wind_s'] = pd.to_numeric(weather['wind_s'])
    weather['time_weather'] = pd.to_datetime(weather['time_weather'])
    
    return weather

In [17]:
def get_arrivals(icao):
    tomorrow = datetime.now().date() + timedelta(days=1)

    url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/"+icao+"/"+str(tomorrow)+"T08:00/"+str(tomorrow)+"T20:00"
    querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

    headers = {
        "X-RapidAPI-Key": "3f18e5fcdbmsh6fb3ed9dda5cb0fp1ddeb7jsndf2a5d517035",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }


    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()

    arrivals = response['arrivals']
    arrivals = pd.json_normalize(arrivals)

    arrivals = arrivals[['departure.airport.name', 'arrival.scheduledTimeUtc', 'arrival.terminal']]

    arrivals.columns = ['airport_name', 'arrival_time_utc', 'terminal']
    arrivals['arrival_time_utc'] = pd.to_datetime(arrivals['arrival_time_utc'])

    return arrivals

In [27]:
def create_connection_string():
    host="wbs1.cqbmtgw3eh6j.eu-central-1.rds.amazonaws.com"        # to connect to your local server
    port=3306

    user="root" 
    password="testdatabase" # your password!!!!

    schema="scooters"   # name of the database you want to use here
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
            
    return con

In [30]:
def get_information_from_cities(con):
    cities = pd.read_sql('SELECT * FROM static_cities', con)

    for index, row in cities.iterrows():
        weather = get_weather(row['city_name'])
        weather['city_id'] = row['city_id']
        weather['ingested_time'] = datetime.now()

        weather.to_sql('weather', con, if_exists='append', index=False)

In [31]:
def get_information_from_airports(con):
    airports = pd.read_sql('SELECT * FROM static_airports', con)

    for index, row in airports.iterrows():
        arrivals = get_arrivals(row['airport_icao'])
        arrivals['airport_icao'] = row['airport_icao']
        arrivals['ingested_time'] = datetime.now()

        arrivals.to_sql('arrivals', con, index=False, if_exists='append')

In [26]:
def main():
    con = create_connection_string()
    get_information_from_cities(con)
    get_information_from_airports(con)


In [24]:
main()